In [1]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torch.utils.data import DataLoader
from torchsummary import summary

from occluded_face.loader import get_train_val_dataset

In [2]:
transforms = {
    'train': transforms.Compose([
        transforms.Resize(112),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225]
        )
    ]),

    'val': transforms.Compose([
        transforms.Resize(112),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225]
        )
    ])
}


In [3]:
train_set, val_set = get_train_val_dataset('./new_train_3/', 0.8, transforms)

In [4]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

In [8]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', weights=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
summary(model, (3, 112, 112))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 56, 56]             864
       BatchNorm2d-2           [-1, 32, 56, 56]              64
             ReLU6-3           [-1, 32, 56, 56]               0
            Conv2d-4           [-1, 32, 56, 56]             288
       BatchNorm2d-5           [-1, 32, 56, 56]              64
             ReLU6-6           [-1, 32, 56, 56]               0
            Conv2d-7           [-1, 16, 56, 56]             512
       BatchNorm2d-8           [-1, 16, 56, 56]              32
  InvertedResidual-9           [-1, 16, 56, 56]               0
           Conv2d-10           [-1, 96, 56, 56]           1,536
      BatchNorm2d-11           [-1, 96, 56, 56]             192
            ReLU6-12           [-1, 96, 56, 56]               0
           Conv2d-13           [-1, 96, 28, 28]             864
      BatchNorm2d-14           [-1, 96,

Using cache found in /home/vv-team/.cache/torch/hub/pytorch_vision_v0.10.0
/home/vv-team/.pyenv/versions/3.8.10/envs/occ/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)

classes = ['not_occluded', 'top_occluded', 'bottom_occluded']
class_count = [6480, 1518, 6899]
weight = 1 / torch.Tensor(class_count)
criterion = torch.nn.MultiLabelSoftMarginLoss(
    weight=weight.to(device)
)

In [ ]:
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    for i, (images, labels) in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        if (i + 1) % 10 == 0:
            outputs = outputs > 0.5
            acc = (outputs == labels).float().mean()
            print(f'Epoch: {epoch + 1}, Iteration: {i + 1}, Loss: {loss.item():.4f}, Accuracy: {acc.item():.4f}')